참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 05.010

### 문제에서 주어진 변수

#### 각 구간 길이

In [ ]:
L_AB_m = sy.symbols('L_{AB}[m]', positive=True)

#### 단면

In [ ]:
b_m, h_m = sy.symbols('b[m], h[m]', positive=True)

#### 하중

In [ ]:
W_N = sy.Symbol('W[N]', real=True)

#### 재료

In [ ]:
sigma_allowed_Pa, tau_allowed_Pa = sy.symbols('sigma_{allowed}[Pa], tau_{allowed}[Pa]', positive=True)

### 분포하중

In [ ]:
w_AB_N_m = W_N / L_AB_m

### 반력

In [ ]:
R_A_N, R_B_N = sy.symbols('R_{A}[N], R_{B}[N]', real=True)

x 좌표

In [ ]:
x_m = sy.Symbol('x[m]', real=True)

분포하중이 A 점에 가하는 모멘트

In [ ]:
M_W_A_Nm = sy.integrate( w_AB_N_m * x_m,(x_m, 0, L_AB_m))
M_W_A_Nm

A점 중심의 모멘트

In [ ]:
M_A_eq = sy.Eq(M_W_A_Nm, R_B_N * L_AB_m)
M_A_eq

반력의 합

In [ ]:
R_eq = sy.Eq(w_AB_N_m * L_AB_m, R_A_N + R_B_N)
R_eq

연립방정식을 풂

In [ ]:
r_sol_list = sy.solve((R_eq, M_A_eq), (R_A_N, R_B_N))
r_sol_list

### 전단력선도

In [ ]:
V_AB_N = R_A_N - w_AB_N_m * x_m
V_AB_N

전단력이 0이 되는 위치

In [ ]:
zero_V_x_m_list = sy.solve(V_AB_N.subs(r_sol_list), x_m)
zero_V_sol = {x_m: zero_V_x_m_list[0]}
zero_V_sol

### 최대 전단응력

* 전단력

In [ ]:
V_N = V_AB_N

* 전체 단면의 I

In [ ]:
I_m4 = (b_m * h_m ** 3) / 12

* $y = 0$ 인 중립면에서 발생할 것이다.

* 중립면 위의 면적

In [ ]:
A_m2 = b_m * h_m / 2

* 중립면 위 면적의 도심과 중립면 사이의 거리

In [ ]:
y_prime_bar_m = h_m/4

* 중립면 위 면적의 모멘트 $Q$

In [ ]:
Q_max_m3 = A_m2 * y_prime_bar_m
Q_max_m3

* $x$ 지점에서의 중립면의 전단응력 eq. 5.8 (p. 199)

In [ ]:
tau_max_Pa = (V_N * Q_max_m3 / (b_m * I_m4)).subs(r_sol_list)
tau_max_Pa

* $x=0$ 지점에서의 중립면 전단응력

In [ ]:
tau_max_Pa = (V_N * Q_max_m3 / (b_m * I_m4)).subs(r_sol_list).subs(x_m, 0)
tau_max_Pa

### 허용 전단 응력을 발생시키는 하중

위에서 구한 중립면 전단응력식과 허용 전단 응력 값 사이의 관계

In [ ]:
tau_eq = sy.Eq(tau_max_Pa, tau_allowed_Pa)
tau_eq

In [ ]:
w_max_shear_list = sy.solve(tau_eq, W_N)
w_max_shear_sol = {W_N:w_max_shear_list[0]}
w_max_shear_sol

### 굽힘모멘트 선도

In [ ]:
M_Nm = (R_A_N * x_m - w_AB_N_m * (x_m**2) / 2).subs(r_sol_list)
M_Nm

### 최대 법선 굽힘응력

p. 170 eq. 5.4a

$$
\sigma_{max} = \frac{M_{max}C}{I}
$$

In [ ]:
sigma_Pa = M_Nm * (h_m / 2) / I_m4
sigma_Pa

$x = 0$, $x = L$ 에서의 응력

In [ ]:
sigma_Pa.subs(x_m, 0), sigma_Pa.subs(x_m, L_AB_m)

전단력이 0인 위치에서의 응력

In [ ]:
sigma_zero_V_Pa = sigma_Pa.subs(zero_V_sol)
sigma_zero_V_Pa

전단력이 0인 위치에서의 응력이 허용 법선응력이 되는 하중

In [ ]:
sigma_eq = sy.Eq(sigma_zero_V_Pa, sigma_allowed_Pa)
sigma_eq

In [ ]:
w_max_bending_list = sy.solve(sigma_eq, W_N)
w_max_bending_sol = {W_N: w_max_bending_list[0]}
w_max_bending_sol

### 임계 L

최대 전단응력과 최대 굽힘응력이 동시에 사용값에 도달할 길이

In [ ]:
span_eq = sy.Eq(w_max_bending_sol[W_N],w_max_shear_sol[W_N])
span_eq

In [ ]:
L_sol=sy.solve(span_eq, L_AB_m)
L_sol

### tests

위 계산이 맞다는 것을 확인하기 위해 어떤 추가 검산이 가능할 것인가?

In [ ]:
assert (0 == sigma_Pa.subs(x_m, 0))

In [ ]:
assert (0 == sigma_Pa.subs(x_m, L_AB_m))